# Testing Parent Hamiltonian Library

Here we test the parent hamiltonian library with different ansatzes that will not have the simmetry of the orginal japanes github one

In [ ]:
#Python libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
# QLM ansatz
import parent_hamiltonian.ansatz.ansatz_qlm as ansatz_qlm
# ParentHamiltonian Complete Version
import parent_hamiltonian.parent.parent_hamiltonian as parent

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()

In [ ]:
# QLM qpus
from qlmaas.qpus import LinAlg, MPS
qpu_qaass = LinAlg()
qpu_mps = MPS(lnnize =True)

In [ ]:
# For QLM observables
from qat.core import Observable, Term

## 1. Japanese Ansatz Generalization

Here we take an ansatz generalization of the japanese used one. The structure is the same but for each qubit the parameters are different. This will break the simetry of the ansatz and the local reduced density matrix for eac qubit will be different!!

This ansatz generalization is in the *ansatz_qlm_general* function into the **parent_hamiltonian/ansatz/ansatz_qlm** package

In [ ]:
nqubit = 9

In [ ]:
# Creating qlm circuit
qprog = ansatz_qlm.ansatz_qlm_general(nqubits=nqubit, depth=3)
circuit = qprog.to_circ()

In [ ]:
%qatdisplay circuit --svg

In [ ]:
# Fixing angles
angle_list = np.random.rand(len(circuit.get_variables()))*2*np.pi 
circuit = circuit(
    ** {v: angle_list[i] for i,v in enumerate(circuit.get_variables())})

In [ ]:
%qatdisplay circuit --svg

Given a QLM circuit from an ansatz the function *solving_ansatz* from **parent_hamiltonian/ansatz/ansatz_qlm** allows to solve the QLM circuit and provides the result as a nqubit-tensor, that will be in the correct form for the *parent_hamiltonian* function from **parent_hamiltonian/parent/parent_hamiltonian**.

The *solving_ansatz* function needs the QLM qpu that we want to use for solving the ansatz.

In [ ]:
# myqlm c lineal algebra
mps_state_c = ansatz_qlm.solving_ansatz(circuit, nqubit, qpu_c)

In [ ]:
mps_state_p= ansatz_qlm.solving_ansatz(circuit, nqubit, qpu_p)
np.isclose(mps_state_c, mps_state_p).all()

In [ ]:
mps_state_ass = ansatz_qlm.solving_ansatz(circuit, nqubit, qpu_qaass)
np.isclose(mps_state_p, mps_state_ass).all()

In [ ]:
mps_state_mps = ansatz_qlm.solving_ansatz(circuit, nqubit, qpu_mps)
np.isclose(mps_state_c, mps_state_mps).all()

In [ ]:
# Computing Parent Hamiltonian terms
h_coefficients, h_paulistrings, h_qubits = parent.parent_hamiltonian(mps_state_c)

In [ ]:
# Solving with Observable
ph_terms = [Term(coef, ps, qb) \
    for coef, ps, qb in zip(h_coefficients, h_paulistrings, h_qubits)
]
ph_observable = Observable(nqubit, pauli_terms=ph_terms)

In [ ]:
job_observable = circuit.to_job('OBS', observable=ph_observable)
ph_results = qpu_c.submit(job_observable)

In [ ]:
ph_results.value

We can use the function *get_local_reduced_matrices* from the package **parent_hamiltonian/parent/parent_hamiltonian** for computing the local reduced density matrices for eac qubit!

In [ ]:
local_qubits, local_rho = parent.get_local_reduced_matrices(mps_state_c)

Following cell compare each local reduced density matrix. As can be seen all are different. For the original japanes ansatz the computed local reduced density matrix were equal for each qubit of the ansatz (see **ParentHamiltonian_03_CompleteVersion**). 

In [ ]:
[np.isclose(
    local_rho[i], local_rho[i+1]).all() for i in range(len(local_rho)-1)]

## 2. Other Ansatzes

In [ ]:
from qat.fermion.circuits import make_ldca_circ, make_general_hwe_circ

In [ ]:
nqubit = 6
depth = 2
lda = make_ldca_circ(nqubit, depth)

In [ ]:
%qatdisplay lda --svg

In [ ]:
angles = np.random.rand(len(lda.get_variables()))*np.pi

In [ ]:
lda = lda(
    ** {v: angles[i] for i,v in enumerate(lda.get_variables())})

In [ ]:
%qatdisplay lda --svg

In [ ]:
lda_mps = ansatz_qlm.solving_ansatz(lda, nqubit, qpu_c)

In [ ]:
lda_coeffs, lda_ps, lda_qbuit = parent.parent_hamiltonian(lda_mps)

In [ ]:
lda_h_terms = [Term(coef, ps, qb) for coef, ps, qb in zip(lda_coeffs, lda_ps, lda_qbuit)]

In [ ]:
lda_observable = Observable(nqbits=nqubit, pauli_terms=lda_h_terms)

In [ ]:
job_lda = lda.to_job('OBS', observable=lda_observable)

In [ ]:
results = qpu_c.submit(job_lda)

In [ ]:
results.value

In [ ]:
gen_ans = make_general_hwe_circ(nqubit, depth)

In [ ]:
%qatdisplay gen_ans --svg

In [ ]:
gen_ans = gen_ans(
    ** {v: angles[i] for i,v in enumerate(gen_ans.get_variables())})

In [ ]:
gen_ans_mps = ansatz_qlm.solving_ansatz(gen_ans, nqubit, qpu_c)

In [ ]:
gen_ans_mps_coeffs, gen_ans_mps_ps, gen_ans_mps_qbuit = parent.parent_hamiltonian(gen_ans_mps)

In [ ]:
gen_ans_h_terms = [Term(coef, ps, qb) for coef, ps, qb in zip(gen_ans_mps_coeffs, gen_ans_mps_ps, gen_ans_mps_qbuit)]

In [ ]:
gen_ans_observable = Observable(nqbits=nqubit, pauli_terms=gen_ans_h_terms)

In [ ]:
job_gen_ans = gen_ans.to_job('OBS', observable=gen_ans_observable)

In [ ]:
results_gen = qpu_c.submit(job_gen_ans)

In [ ]:
results_gen.value